# Database Construction

<br>

### Imports

In [1]:
#exports
import json
import pandas as pd

import os
from warnings import warn

In [2]:
from IPython.display import JSON

<br>

### User Inputs

In [3]:
raw_data_dir = '../data/raw'
definitions_dir = '../data/definitions'
intermediate_data_dir = '../data/intermediate'

<br>

### Constructing the Database

We'll begin by loading in and combining the definitions of each source

In [4]:
#exports
def load_source_definitions(definitions_dir):
    source_definitions = dict()
    source_def_filenames = [f for f in os.listdir(definitions_dir) if '.json' in f and f != 'outputs.json']

    for filename in source_def_filenames:
        source_name = filename.replace('.json', '')

        with open(f'{definitions_dir}/{filename}', 'r') as f:
            source_definitions[source_name] = json.load(f)
            
    return source_definitions

In [5]:
source_definitions = load_source_definitions(definitions_dir)
        
JSON(source_definitions)

<IPython.core.display.JSON object>

In [6]:
#exports
def identify_primary_keys(source_definitions):
    primary_keys = []

    for source in source_definitions.values():
        primary_keys += list(source['key_map'].keys())

    primary_keys = sorted(set(primary_keys))
    
    return primary_keys

In [7]:
primary_keys = identify_primary_keys(source_definitions)
    
primary_keys[:5]

['10000', '10001', '10002', '10003', '10004']

In [8]:
#exports
def check_source_for_disallowed_cols(attr_cols, outputs):
    output_cols_to_be_added = [attr_col['output_col'] for attr_col in attr_cols.values()]
    output_cols_allowed = outputs.keys()

    disallowed_output_cols = list(set(output_cols_to_be_added) - set(output_cols_allowed))

    assert len(disallowed_output_cols)==0, f"The following columns are not allowed in the output dataset: {', '.join(disallowed_output_cols)}"
    
def check_sources_for_disallowed_cols(source_definitions, definitions_dir):
    with open(f'{definitions_dir}/outputs.json', 'r') as f:
        outputs = json.load(f)
    
    for source, definition in source_definitions.items():
        check_source_for_disallowed_cols(definition['attr_cols'], outputs)
        
    return

In [9]:
check_sources_for_disallowed_cols(source_definitions, definitions_dir)

In [10]:
#exports
def identify_inputs_for_output_cols(source_definitions):
    keys_to_keep = ['output_col', 'output_rank']
    output_cols = list()
    inputs_for_output_cols = dict()
    
    for source, definition in source_definitions.items():
        for input_col, value in definition['attr_cols'].items():
            output_cols += [(source, input_col, value['output_col'], value['output_rank'])]
            
    df_output_cols = pd.DataFrame(output_cols, columns=['source', 'input_col', 'output_col', 'output_rank'])
    
    for output_col in df_output_cols['output_col']:
        inputs_for_output_cols[output_col] = (df_output_cols
                                              .query('`output_col`==@output_col')
                                              .sort_values('output_rank', ascending=False)
                                              [['source', 'input_col']]
                                              .apply(tuple, axis=1)
                                              .to_list()
                                             )
    return inputs_for_output_cols

In [11]:
inputs_for_output_cols = identify_inputs_for_output_cols(source_definitions)

JSON(inputs_for_output_cols)

<IPython.core.display.JSON object>

In [41]:
def get_input_key_to_attr(df_input, key_input_col, input_col):
    df_key_input_non_duped = df_input[[key_input_col, input_col]].drop_duplicates()

    one_to_many_input_keys = list(df_key_input_non_duped
                                  [df_key_input_non_duped[key_input_col].duplicated(keep=False)]
                                  .sort_values(key_input_col)
                                  [key_input_col]
                                  .unique()
                                 )

    warn(f"The following {key_input_col} input keys were not unique w.r.t. {input_col}, a one to many relationship will therefore be used: {', '.join(one_to_many_input_keys)}\n")

    # Creating one-to-one mapping
    input_key_to_attr = (df_key_input_non_duped
                         [~df_key_input_non_duped['bmu_root'].isin(one_to_many_input_keys)]
                         .set_index(key_input_col)
                         [input_col]
                         .to_dict()
                        )
    
    # Creating one-to-many mapping
    for input_key in one_to_many_input_keys:
        many_inputs = (df_key_input_non_duped
                       .query(f'{key_input_col}=="{input_key}"')
                       [input_col]
                       .to_list()
                      )

        input_key_to_attr[input_key] = ', '.join([str(elem) for elem in many_inputs])
    
    return input_key_to_attr

In [42]:
source = 'ESAIL'
key_input_col = 'bmu_root'
input_col = 'primary_fuel_type'
filename = 'ESAIL.csv'

df_input = pd.read_csv(f"{raw_data_dir}/{filename}")
input_key_to_attr = get_input_key_to_attr(df_input, key_input_col, input_col)

JSON([input_key_to_attr])

<ipython-input-41-1a2f88dfe0a6>:11: UserWarning: The following bmu_root input keys were not unique w.r.t. primary_fuel_type, a one to many relationship will therefore be used: FAWLG, GRAI

  warn(f"The following {key_input_col} input keys were not unique w.r.t. {input_col}, a one to many relationship will therefore be used: {', '.join(one_to_many_input_keys)}\n")


<IPython.core.display.JSON object>

In [43]:
#exports
def get_primary_key_to_attr(source, input_col, source_definitions, raw_data_dir):
    filename = source_definitions[source]['filename']
    key_input_col = source_definitions[source]['key_input_col']
    key_map = source_definitions[source]['key_map']
    value_map = source_definitions[source]['attr_cols'][input_col]['value_map']
    
    df_input = pd.read_csv(f"{raw_data_dir}/{filename}")
    
    if value_map is not None:
        df_input[input_col].isin(value_map.keys()).mean()==1, f'A mapping was provided for {input_col} but did not contain all of the values found within it'
        df_input[input_col] = df_input[input_col].map(value_map)
    
    input_key_to_attr = get_input_key_to_attr(df_input, key_input_col, input_col)

    primary_key_to_attr = (pd.Series(key_map)
                           .map(input_key_to_attr)
                           .to_dict())
    
    return primary_key_to_attr

In [44]:
source = 'ESAIL'
input_col = 'primary_fuel_type'

primary_key_to_attr = get_primary_key_to_attr(source, input_col, source_definitions, raw_data_dir)

JSON([primary_key_to_attr])

<ipython-input-41-1a2f88dfe0a6>:11: UserWarning: The following bmu_root input keys were not unique w.r.t. primary_fuel_type, a one to many relationship will therefore be used: FAWLG, GRAI

  warn(f"The following {key_input_col} input keys were not unique w.r.t. {input_col}, a one to many relationship will therefore be used: {', '.join(one_to_many_input_keys)}\n")


<IPython.core.display.JSON object>

In [45]:
#exports
def update_df_col(df, update_col, update_dict):
    s_update = pd.Series(df.index.map(update_dict), df.index).dropna()
    df.loc[s_update.index, update_col] = s_update.values

    return df

def add_attr_cols(df, source_definitions, raw_data_dir):
    inputs_for_output_cols = identify_inputs_for_output_cols(source_definitions)
    
    for output_col, inputs in inputs_for_output_cols.items():
        for (source, input_col) in inputs:
            primary_key_to_attr = get_primary_key_to_attr(source, input_col, source_definitions, raw_data_dir)
            df = update_df_col(df, output_col, primary_key_to_attr)
            
    return df

In [46]:
df = pd.DataFrame(index=primary_keys)
df = add_attr_cols(df, source_definitions, raw_data_dir)

df.head()

<ipython-input-41-1a2f88dfe0a6>:11: UserWarning: The following bmu_root input keys were not unique w.r.t. primary_fuel_type, a one to many relationship will therefore be used: FAWLG, GRAI

  warn(f"The following {key_input_col} input keys were not unique w.r.t. {input_col}, a one to many relationship will therefore be used: {', '.join(one_to_many_input_keys)}\n")
<ipython-input-41-1a2f88dfe0a6>:11: UserWarning: The following bmu_root input keys were not unique w.r.t. sett_bmu_id, a one to many relationship will therefore be used: ABTH, ABTHG, BAGE, BANGE, BLLA, BRWE, CARR, CNQPS, COCK, COTPS, COWE, CRUA, CRYRW, DDGNO, DICI, DIDC, DIDCB, DIDCG, DINO, DNGB, DPGEN, DRAXX, DRAXXG, DRKPS, DUNG, EGGPS, ERRO, FARR, FASN, FAWL, FAWLG, FERR, FERRG, FFES, FIDL, FIDLG, FIFO, FOYE, GANGE, GANW, GNFSW, GRAI, GRAIG, GRGBW, GRIFW, GYMR, GYMRW, HEYM, HINB, HMGTO, HMRPS, HRTL, HUNB, IRNPS, KILLPG, KINO, KINOG, LARYW, LITTD, LITTDG, LNCSW, LOAN, LYNE, MARK, MPGEN, OLDS, OMNDW, PEHE, PEHEG, PEMB, PPGEN, 

,fuel_type,sett_bmu_id,longitude,latitude
10000,biomass,"E_MARK-1, E_MARK-2",-3.60352,57.4804
10001,coal,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3",-1.26757,51.6236
10002,coal,"T_ABTH7, T_ABTH8, T_ABTH9",-3.40487,51.3873
10003,coal,"T_COTPS-1, T_COTPS-2, T_COTPS-3, T_COTPS-4",-0.648193,53.2455
10004,coal,"T_DRAXX-1, T_DRAXX-2, T_DRAXX-3, T_DRAXX-4, T_...",-0.626221,53.7487


In [47]:
#exports
def add_key_cols(df, source_definitions):
    for source in source_definitions.keys():
        key_map = source_definitions[source]['key_map']
        key_output_col = source_definitions[source]['key_output_col']
        df = update_df_col(df, key_output_col, key_map)
        
    return df

In [48]:
df = pd.DataFrame(index=primary_keys)
df = add_key_cols(df, source_definitions)

df.head()

,bmu_root
10000,MARK
10001,DIDC
10002,ABTH
10003,COTPS
10004,DRAXX


In [49]:
#exports
def construct_output_df(source_definitions, raw_data_dir):
    primary_keys = identify_primary_keys(source_definitions)
    
    df = (pd.DataFrame(index=primary_keys)
          .pipe(add_key_cols, source_definitions)
          .pipe(add_attr_cols, source_definitions, raw_data_dir)
         )
    
    df.index.name = 'osuked_id'
    
    return df

In [50]:
df = construct_output_df(source_definitions, raw_data_dir)

df.head()

<ipython-input-41-1a2f88dfe0a6>:11: UserWarning: The following bmu_root input keys were not unique w.r.t. primary_fuel_type, a one to many relationship will therefore be used: FAWLG, GRAI

  warn(f"The following {key_input_col} input keys were not unique w.r.t. {input_col}, a one to many relationship will therefore be used: {', '.join(one_to_many_input_keys)}\n")
<ipython-input-41-1a2f88dfe0a6>:11: UserWarning: The following bmu_root input keys were not unique w.r.t. sett_bmu_id, a one to many relationship will therefore be used: ABTH, ABTHG, BAGE, BANGE, BLLA, BRWE, CARR, CNQPS, COCK, COTPS, COWE, CRUA, CRYRW, DDGNO, DICI, DIDC, DIDCB, DIDCG, DINO, DNGB, DPGEN, DRAXX, DRAXXG, DRKPS, DUNG, EGGPS, ERRO, FARR, FASN, FAWL, FAWLG, FERR, FERRG, FFES, FIDL, FIDLG, FIFO, FOYE, GANGE, GANW, GNFSW, GRAI, GRAIG, GRGBW, GRIFW, GYMR, GYMRW, HEYM, HINB, HMGTO, HMRPS, HRTL, HUNB, IRNPS, KILLPG, KINO, KINOG, LARYW, LITTD, LITTDG, LNCSW, LOAN, LYNE, MARK, MPGEN, OLDS, OMNDW, PEHE, PEHEG, PEMB, PPGEN, 

,bmu_root,fuel_type,sett_bmu_id,longitude,latitude
osuked_id,,,,,
10000,MARK,biomass,"E_MARK-1, E_MARK-2",-3.60352,57.4804
10001,DIDC,coal,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3",-1.26757,51.6236
10002,ABTH,coal,"T_ABTH7, T_ABTH8, T_ABTH9",-3.40487,51.3873
10003,COTPS,coal,"T_COTPS-1, T_COTPS-2, T_COTPS-3, T_COTPS-4",-0.648193,53.2455
10004,DRAXX,coal,"T_DRAXX-1, T_DRAXX-2, T_DRAXX-3, T_DRAXX-4, T_...",-0.626221,53.7487


In [51]:
df.to_csv(f'{intermediate_data_dir}/power_stations.csv')